In [1]:
import time
from termcolor import colored
import trio

base_time = time.monotonic()

async def do_a_job(t, label, nursery=None):
    print(colored(f"starting a {t} second job, '{label}' at time {time.monotonic()-base_time:.1f}", "green"))
    if nursery is not None:
        nursery.start_soon(do_a_job, t*1.5, label+'.child')
    # if t in {5, 6}:
    #     raise Exception("can't run a job for 5 or 6 seconds")
    await trio.sleep(t)
    print(colored(f"finished a {t} second job, '{label}' at time {time.monotonic()-base_time:.1f}", "red"))
    
async def do_many_jobs():
    print(f"starting many jobs at time {time.monotonic()-base_time:.1f}")
    with trio.move_on_after(80) as cancel_scope:
        async with trio.open_nursery() as nursery:
            nursery.start_soon(do_a_job, 4, 'A')
            nursery.start_soon(do_a_job, 2, 'B')
            nursery.start_soon(do_a_job, 5, 'C', nursery)
            nursery.start_soon(do_a_job, 1, 'D', nursery)
            nursery.start_soon(do_a_job, 6, 'E')
    if cancel_scope.cancelled_caught:
        print(f"may have finished some jobs at time {time.monotonic()-base_time:.1f}")
    else:
        print(f"finished many jobs at time {time.monotonic()-base_time:.1f}")
    
async def main():
    await do_many_jobs()

trio.run(main)

starting many jobs at time 0.0
starting a 6 second job, 'E' at time 0.0
starting a 5 second job, 'C' at time 0.0
starting a 4 second job, 'A' at time 0.0
starting a 2 second job, 'B' at time 0.0
starting a 1 second job, 'D' at time 0.0
starting a 1.5 second job, 'D.child' at time 0.0
starting a 7.5 second job, 'C.child' at time 0.0
finished a 1 second job, 'D' at time 1.0
finished a 1.5 second job, 'D.child' at time 1.5
finished a 2 second job, 'B' at time 2.0
finished a 4 second job, 'A' at time 4.0
finished a 5 second job, 'C' at time 5.0
finished a 6 second job, 'E' at time 6.0
finished a 7.5 second job, 'C.child' at time 7.5
finished many jobs at time 7.5
